# Glaze paper notes
[Glaze](https://arxiv.org/pdf/2302.04222.pdf) 

- perceptual pertubation budget: `p`
- impact of input perturbation: `alpha`
- Given artwork `x`
- feature exterator `phi`
- style-transferred version of x into target style `T:omega(x,T)`
- Style cloak `delta_x`
- Stable Diffusion provides both `epsilon` and `theta`

![Alt text](figures/glaze_eq1.png)

1) choose target style that is difrent than og artist style
2) transfer og artists images to new style using style transfer
3) compute cloak pertubation using [LPIPS](https://github.com/richzhang/PerceptualSimilarity)

![Alt text](figures/glaze_eq2.png)


# Implementing glaze paper
[HuggingFace stable diffusion pipleine](https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py)

### Feature extractor
[CLIP](https://huggingface.co/docs/transformers/model_doc/clip)

[image_processing_clip](https://github.com/huggingface/transformers/blob/v4.26.1/src/transformers/models/clip/image_processing_clip.py#L45)

In [ ]:
%pip install Pillow
%pip install transformers
%pip install torch torchvision
%pip install clip
%pip install matplotlib
%pip install lpips

In [ ]:
import torch
import torchvision.transforms as transforms
from clip import clip
import matplotlib.pyplot as plt
import torchvision.utils as vutils

# Load CLIP feature extractor and model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device)
model.eval()

# Load images A and B + perturbation
img_a = preprocess(Image.open('dog.png')).unsqueeze(0).to(device)
img_b = preprocess(Image.open('cow.png')).unsqueeze(0).to(device)
perturbation = torch.randn_like(img_b, requires_grad=True)

# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam([perturbation], lr=0.01)

# Loop to adjust perturbation and minimize loss
for i in range(1000):
    # Compute features for img_a and img_b + perturbation
    features_a = model.encode_image(img_a)
    features_b = model.encode_image(img_b + perturbation)

    # Compute loss
    loss = criterion(features_a, features_b)
    # loss = torch.norm(features_a - features_b, p=2)

    # Zero gradients, compute gradients, and update perturbation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Step {i}: Loss = {loss.item()}')

# Apply perturbation to img_b and save perturbed image
perturbed_img_b = (img_b + perturbation).squeeze(0).cpu()
#transforms.ToPILImage()(perturbed_img_b).save('path/to/perturbed_image_b.jpg')
# Convert the tensor image to a NumPy array
img_array = perturbed_img_b.detach().numpy().transpose(1, 2, 0)

# Display the image using matplotlib
plt.imshow(img_array)
plt.axis('off')
plt.show()

In [ ]:
import lpips
import torch
import torchvision.transforms as transforms
from clip import clip
import matplotlib.pyplot as plt

# Load CLIP feature extractor and model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device)
model.eval()

# Load images A and B + perturbation
img_a = preprocess(Image.open('dog.png')).unsqueeze(0).to(device)
img_b = preprocess(Image.open('cow.png')).unsqueeze(0).to(device)
perturbation = torch.randn_like(img_b, requires_grad=True)

# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam([perturbation], lr=0.01)

# Define penalty weight and lpips loss function
penalty_weight = 0.1
p = 0.05
lpips_loss = lpips.LPIPS(net='vgg').to(device)

# Loop to adjust perturbation and minimize loss
for i in range(1000):
    # Compute features for img_a and img_b + perturbation
    features_a = model.encode_image(img_a)
    features_b = model.encode_image(img_b + perturbation)

    # Compute loss
    loss = torch.norm(features_a - features_b, p=2) + penalty_weight * lpips_loss(perturbation - p, torch.zeros_like(perturbation))

    # Zero gradients, compute gradients, and update perturbation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Step {i}: Loss = {loss.item()}')

# Apply perturbation to img_b and save perturbed image
perturbed_img_b = (img_b + perturbation).squeeze(0).cpu()
vutils.save_image(perturbed_img_b, 'cow_dog.png')
# transforms.ToPILImage()(perturbed_img_b).save('perturbed_image_b.png')
# Convert the tensor image to a NumPy array
img_array = perturbed_img_b.detach().numpy().transpose(1, 2, 0)

# Display the image using matplotlib
plt.imshow(img_array)
plt.axis('off')
plt.show()